In [1]:
import pandas as pd
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    pipeline, DataCollatorForSeq2Seq, EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict
import torch
import os

!pip install sacrebleu
import sacrebleu

os.environ["WANDB_DISABLED"] = "true"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.4 MB/s eta 0:00:00


In [2]:
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
data = pd.read_csv("/kaggle/input/english-nagamese-raw-data/raw_data.csv")
data = data[:100]
data.rename(columns={"English": "en", "Nagamese": "nagamese"}, inplace=True)
dataset = Dataset.from_pandas(data)

In [4]:
# Split dataset into train and validation sets
train_test_split = dataset.train_test_split(test_size=0.2, seed=40)
dataset = DatasetDict({
    "train": train_test_split["train"],
    "validation": train_test_split["test"]
})


In [5]:
# Load Pretrained NLLB Model and Tokenizer
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [6]:
# Modify dropout rates (Joint Dropout)
model.config.dropout = 0.3  # Dropout rate for the whole model
model.config.attention_dropout = 0.3  # Dropout rate for attention layers
model.config.activation_dropout = 0.3  # Dropout in activation functions
model.config.encoder_layerdrop = 0.3  # Dropout in encoder layers
model.config.decoder_layerdrop = 0.3  # Dropout in decoder layers

In [7]:
# Add a new language token for Nagamese
new_lang_token = "__nag__"

# Ensure the token is added to the tokenizer
tokenizer.add_tokens([new_lang_token])  # Add Nagamese token to tokenizer


# Define source and target languages
tgt_lang = "__eng_Latn__"  
src_lang = new_lang_token  

tokenizer.src_lang = src_lang
tokenizer.tgt_lang = tgt_lang

In [8]:
# Tokenize Dataset
def preprocess_function(examples):
    inputs = examples["nagamese"]
    targets = examples["en"]
    model_inputs = tokenizer(inputs, text_target=targets, truncation=True, max_length=128)
    return model_inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [9]:
# Check the dataset structure
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['en', 'nagamese', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['en', 'nagamese', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20
    })
})


In [10]:
# Define Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps", 
    save_strategy="steps",
    eval_steps=50,
    save_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    warmup_steps=200,
    seed = 40,
    weight_decay=0.01,
    save_total_limit=6,
    num_train_epochs=20,
    predict_with_generate=True,
    optim="adafactor", 
    report_to=[],
    fp16=True,
    logging_dir="./logs",
    logging_steps=50,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

# Start Training
trainer.train()

<ipython-input-10-b04a060e1e5d>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=40, training_loss=3.5205047607421873, metrics={'train_runtime': 55.5171, 'train_samples_per_second': 28.82, 'train_steps_per_second': 0.72, 'total_flos': 161584736894976.0, 'train_loss': 3.5205047607421873, 'epoch': 13.4})

In [11]:
# Save the Fine-tuned Model
model.save_pretrained("./fine_tuned_nllb")
tokenizer.save_pretrained("./fine_tuned_nllb")


('./fine_tuned_nllb/tokenizer_config.json',
 './fine_tuned_nllb/special_tokens_map.json',
 './fine_tuned_nllb/sentencepiece.bpe.model',
 './fine_tuned_nllb/added_tokens.json',
 './fine_tuned_nllb/tokenizer.json')

In [12]:
# Translation Pipeline
translator = pipeline(
    "translation",
    model="./fine_tuned_nllb",
    tokenizer="./fine_tuned_nllb",
    device=0 if torch.cuda.is_available() else -1  
)


# Test Translation
translated = translator(
    "How are you?",
    tgt_lang="eng_Latn",  
    src_lang="nag"        
)
print(translated)

Device set to use cuda:0


[{'translation_text': 'How are you?'}]


In [13]:
!pip install transformers sacrebleu
import os
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import sacrebleu
import pandas as pd

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [15]:
model = model.to(device)

In [16]:
# Extract source and reference texts from the validation set
source_texts = dataset["validation"]["nagamese"]  # English source texts
reference_texts = dataset["validation"]["en"]  # Nagamese target texts

# Check the extracted texts
print("Source Texts (Nagamese):", source_texts[:5])
print("Reference Texts (English):", reference_texts[:5])


Source Texts (Nagamese): ['Paul aru Silvanus aru Timothy pora Isor aru Probhu Jisu Khrista te thaka Thessalonica laga girja ke likhi ase. Anugrah aru shanti apnikhan uporte hobi.', 'Isor agete apnikhan nimite amikhan laga khushi thaka to ki pora Isor ke dhanyabad dibo?', 'Aru jitia Probhu Jisu tai laga pobitro manu khan logote ahibo, titia Baba Isor usor te apnikhan laga mon ke takot dibo aru eku dagi nathaka aru pobitrota te thakibo nimite. Amen!', 'Itu nimite, ami laga bhai khan, mon dukh hoile bi apnikhan laga biswas nimite amikhan laga mon aram hoise.', 'Sob saman ke bhal pora bujhi bi. Ki bhal ase itu dhuri kena thakibi.']
Reference Texts (English): ['Paul, Silvanus, and Timothy to the church of the Thessalonians in God the Father and the Lord Jesus Christ. May grace and peace be to you.', 'For what thanks can we give to God for you, for all the joy that we have before our God over you?', 'May he strengthen your hearts so that they will be blameless in holiness before our God and 

In [17]:
# Ensure tokenizer recognizes `__nag__` for forced decoding
model.eval()
machine_translations = []

for text in source_texts:
    inputs = tokenizer(text, return_tensors="pt", max_length=64, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang))
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    machine_translations.append(translated_text)


In [18]:
# Compute BLEU Score
wrapped_reference_texts = [reference_texts]  # Ensure correct format
bleu = sacrebleu.corpus_bleu(machine_translations, wrapped_reference_texts)
print("BLEU Score:", bleu.score)

BLEU Score: 1.396126382334122
